## hate speech classification demo

In this notebook we will be testing out our model with some random sample tweets. You'll need to [download our model](https://drive.google.com/drive/folders/1UE1MiiNbXWgalJ1UIDW7mTLsSH15UIfX?usp=sharing) and place it in the current directory.

## installation

In [2]:
%pip install --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint
%pip install torch
%pip install transformers

  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to /private/var/folders/j_/ygb_gxx970b1bfpk3v12gc980000gp/T/pip-install-1z0ezn9l/twint_42504afc583c4bffb21a62b3b3dbd21a
  Using cached dataclasses-0.6-py3-none-any.whl (14 kB)
Using legacy 'setup.py install' for twint, since package 'wheel' is not installed.
  Attempting uninstall: twint
    Found existing installation: twint 2.1.20
    Uninstalling twint-2.1.20:
      Successfully uninstalled twint-2.1.20
    Running setup.py install for twint ... done
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.5/libexec/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.5/libexec/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.5/libexec/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## imports

In [3]:
import torch
import transformers
import twint
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, AdamW, DistilBertConfig
import nest_asyncio

Let's load our model and tokenizer.

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
model = torch.load('model.bin', map_location=device)

In [6]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Now, let's fetch some tweets and clean them up.

In [7]:
nest_asyncio.apply() # Asyncronous running for twint to work in this notebook

c = twint.Config()

c.Custom["tweet"] = ["id"]
c.Custom["user"] = ["bio"]
c.Limit = 10
c.Store_csv = True
c.Output = "none"

twint.run.Search(c)

CRITICAL:root:twint.run:Twint:Feed:noData'globalObjects'
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noData'globalObjects'
sleeping for 8.0 secs


KeyboardInterrupt: 